# Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning

## A primer in machine learning

- traditional programming: rule + data => answers
- machine learning: answers + data => rule

## Hello world example 

- How would you infer the equation like below by training neural network on input data?
```
float hw_function(float x){
    float y = (2 * x) - 1;
    return y;
}
```

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# define and compile the neural network
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

# provide the data
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# train the neural network
model.fit(xs, ys, epochs=500)
print(model.predict([10.0]))

## Computer vision example

In [ ]:
import tensorflow as tf
print(tf.__version__)

# the fashion MNIST data is available directly in the tf.keras datasets API
mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
# observe how values look like
import numpy as np
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
plt.imshow(training_images[0])
print(training_labels[0])
print(training_images[0])

In [ ]:
# normalize
training_images  = training_images / 255.0
test_images = test_images / 255.0

# define the model
# Flatten: turns matrix into a 1 dimensional set
# Dense: Adds a layer of neurons (each layer of neurons need an activation function)
# Relu: "If X>0 return X, else return 0" -- it only passes values 0 or greater to the next layer in the network
# Softmax: takes a set of values, and effectively picks the biggest one
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# buld the model by compiling it with an optimizer and loss function
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train the model by asking it to fit your training data to your training labels
model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

## Callback

In [ ]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.6):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

callbacks = myCallback()

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])